# Rate My Professor Web Scaping Workshop 4/19/2023 for CDJ

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import csv

# most of these are disabling/closing popups
options = Options()
options.add_argument("--disable-notifications")
options.add_argument("start-maximized")
options.add_argument("enable-automation")
options.add_argument("--no-sandbox")
options.add_argument("--disable-infobars")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-browser-side-navigation")
options.add_argument("--disable-gpu"); 
options.add_argument("--disable-popup-blocking");
#cmd-shift-g
# For Mac: ~/Library/Application Support/Google/Chrome/Default/Extensions
path_to_extension = r'~/Library/Application Support/Google/Chrome/Default/Extensions/cfhdojbkjhnklbpkdaibdccddilifddb/3.16.2_0'
options.add_argument('load-extension=' + path_to_extension)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [125]:
driver.get("https://www.ratemyprofessors.com/search/teachers?query=*&sid=298")

In [126]:
# using the wait command to close a cookie popup that the previous options didn't account for
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='Buttons__Button-sc-19xdot-1 CCPAModal__StyledCloseButton-sc-10x9kq-2 gvGrz']"))).click()

In [127]:
#get more of list
num = 2557
# button_clicks = (int(num) - 8)/8
button_clicks = 8

for i in range(int(button_clicks)):
    button = driver.find_element("xpath","//button[@class = 'Buttons__Button-sc-19xdot-1 PaginationButton__StyledPaginationButton-txi1dr-1 gjQZal']")
    button.click()
    time.sleep(2)

In [128]:
#getting dictionary of professor urls

prof_track = {}
professors = driver.find_elements("xpath", "//a[@class = 'TeacherCard__StyledTeacherCard-syjs0d-0 dLJIlx']")

for x in professors:
    name = x.find_element("xpath", ".//div[@class ='CardName__StyledCardName-sc-1gyrgim-0 cJdVEK']").get_attribute("innerText")
    url = x.get_attribute("href")
    prof_track.update( {name : url} )

In [129]:
def get_key_from_value(d, val):
    return [k for k, v in d.items() if v == val]

In [133]:
rmp_dict = {"Professor":[], "Department":[], "Content":[], "Quality_Score":[],"Difficulty_Score":[]}

for url in prof_track.values():
    driver.get(url)

        # printing out the overall number of ratings for each professor.
    num_ratings = driver.find_element("xpath","//html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[1]/div[2]/div/a").text.split(" ")[0]
        # print(num_ratings)

        # calcuate how many load more buttons we need to click
    button_clicks = round((int(num_ratings) - 20)/10)
    #print(button_clicks)

    for i in range(int(button_clicks)):
        button = driver.find_element("xpath","//button[@class = 'Buttons__Button-sc-19xdot-1 PaginationButton__StyledPaginationButton-txi1dr-1 gjQZal']")
        button.click()
        time.sleep(1)

    reviews = driver.find_elements("xpath","//div[@class ='Rating__StyledRating-sc-1rhvpxz-1 jcIQzP']")

    department = driver.find_element("xpath",".//a[@class='TeacherDepartment__StyledDepartmentLink-fl79e8-0 fuypDB']").get_attribute("innerText")

    for review in reviews:
        content = review.find_element("xpath", ".//div[@class ='Comments__StyledComments-dzzyvm-0 gRjWel']").get_attribute("innerText")
        # print(content)
        quality_score = review.find_element("xpath", ".//div[@class ='RatingValues__StyledRatingValues-sc-6dc747-0 gFOUvY']").get_attribute("innerText").splitlines()[1]
        #print(quality_score)
        difficulty_score = review.find_element("xpath", ".//div[@class ='RatingValues__StyledRatingValues-sc-6dc747-0 gFOUvY']").get_attribute("innerText").splitlines()[3]
        #print(difficulty_score)

        rmp_dict["Professor"].append(get_key_from_value(prof_track, url))
        rmp_dict["Department"].append(department)
        rmp_dict["Content"].append(content)
        rmp_dict["Quality_Score"].append(quality_score)
        rmp_dict["Difficulty_Score"].append(difficulty_score)

In [135]:
#rmp_dict

In [139]:
df = pd.DataFrame.from_dict(rmp_dict)

In [143]:
df.tail()

,Professor,Department,Content,Quality_Score,Difficulty_Score
1755,[Deborah Streeter],Business department,The best of the best!,5.0,3.0
1756,[Deborah Streeter],Business department,this class is excellent if you're interested i...,5.0,4.0
1757,[Deborah Streeter],Business department,No Comments,1.0,3.0
1758,[Deborah Streeter],Business department,"Dedicated, inspiring and creative. You'll want...",5.0,2.0
1759,[Deborah Streeter],Business department,No Comments,5.0,3.0
